In [1]:
# ===== Include all needed libraries / packages =====


import os
import pandas as pd
import numpy as np

In [2]:
# Check / Create needed folder structure


list_folder_structure = [
    "output_csv_data",
    "output_plot/axial_ratio_theta_vs_phi",
    "output_plot/axial_ratio_theta_vs_freq",
    "output_plot/gain_total_theta_vs_phi",
    "output_plot/gain_total_theta_vs_freq",
    "output_plot/gain_lhcp_theta_vs_phi",
    "output_plot/gain_lhcp_theta_vs_freq",
    "output_plot/gain_rhcp_theta_vs_phi",
    "output_plot/gain_rhcp_theta_vs_freq"
]

for directory_path in list_folder_structure:
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

In [3]:
# ===== Create list of frequencies & dataframes from csv files =====


# Specify the directory where the CSV files are
directory = './input_csv_data/'

# Specify a list of all CSV filenames in the directory
csv_files = [
        '4panneaux-R-corr-CL-circ-norme_0.139000GHz.csv',
        '4panneaux-R-corr-CL-circ-norme_0.140000GHz.csv',
        '4panneaux-R-corr-CL-circ-norme_0.141000GHz.csv',
        '4panneaux-R-corr-CL-circ-norme_0.142000GHz.csv',
        '4panneaux-R-corr-CL-circ-norme_0.143000GHz.csv',
        '4panneaux-R-corr-CL-circ-norme_0.144000GHz.csv',
        '4panneaux-R-corr-CL-circ-norme_0.145000GHz.csv'
    ]

# Create a list of frequencies (int)
list_of_freq_mhz = np.arange(139, 146)

# Create a list of DataFrames
list_of_dfs = [pd.read_csv(os.path.join(directory, f)) for f in csv_files]


In [4]:
# ===== Calculate Weight =====


for df in list_of_dfs:
    df["Weight"] = np.abs(np.sin(np.radians(df.Theta)))

In [5]:
# ===== Calculate Linear Gain for RHCP / LHCP =====


for df in list_of_dfs:
    df["Gain_RHCP_Linear"] = 10 ** (df.Gain_RHCP/10)
    df["Gain_LHCP_Linear"] = 10 ** (df.Gain_LHCP/10)
    
    df["Gain_LHCP_Weight"] = df.Gain_LHCP * df.Weight
    df["Gain_RHCP_Weight"] = df.Gain_RHCP * df.Weight

In [6]:
# ===== Calculate Axial Ratio =====


for df in list_of_dfs:
    gain_rhcp_linear_tmp_for_efield_axial_ratio = 10 ** (df.Gain_RHCP/20)
    gain_lhcp_linear_tmp_for_efield_axial_ratio = 10 ** (df.Gain_LHCP/20)
    df["Axial_Ratio_Linear"] = abs(gain_rhcp_linear_tmp_for_efield_axial_ratio + gain_lhcp_linear_tmp_for_efield_axial_ratio)/abs(gain_rhcp_linear_tmp_for_efield_axial_ratio - gain_lhcp_linear_tmp_for_efield_axial_ratio)
    df["Axial_Ratio"] = 10 * np.log10(df.Axial_Ratio_Linear)
    df["Axial_Ratio_Weight"] = df.Axial_Ratio * df.Weight

In [7]:
# ===== Calculate Total Gain =====


for df in list_of_dfs:
    df["Gain_Total_Linear"] = abs(df.Gain_RHCP_Linear + df.Gain_LHCP_Linear)
    df["Gain_Total"] = 10 * np.log10(df.Gain_Total_Linear)
    df["Gain_Total_Linear_Weight"] = df.Gain_Total_Linear * df.Weight
    df["Gain_Total_Weight"] = df.Gain_Total * df.Weight

In [8]:
# ===== Calculate Total Gain (with offset from Phaseshifter) =====


ps_offset_db = 2.4

for df in list_of_dfs:
    df["Gain_Total_Offset"] = df.Gain_Total + ps_offset_db
    df["Gain_Total_Offset_Linear"] = 10 ** (df.Gain_Total_Offset / 10)
    df["Gain_Total_Offset_Linear_Weight"] = df.Gain_Total_Offset_Linear * df.Weight
    df["Gain_Total_Offset_Weight"] = df.Gain_Total_Offset * df.Weight

In [9]:
# ===== Export to csv files with new data =====


for index, df in enumerate(list_of_dfs):
    file_path = 'output_csv_data/data_' + str(list_of_freq_mhz[index]) + 'mhz.csv'
    df.to_csv(file_path, index=False)

In [10]:
print("Done")

Done
